Flip axis in MR images

In [ ]:
import os
import nibabel as nib
import json

input_folder = '/mnt/sda1/Repos/a-eye/Output/mri_qc/samples/samples_v3_bids'
output_folder = '/home/jaimebarranco/Desktop/tmp/test_inference_flipped_images/flipped_images'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Function to flip the axis of the image without flipping the image itself
def flip_axis(img):
    data = img.get_fdata()
    flipped_data = data[::-1, :, :]
    flipped_img = nib.Nifti1Image(flipped_data, img.affine, img.header)
    return flipped_img

# Iterate through the BIDS folder to find NIfTI and JSON files
for root, dirs, files in os.walk(input_folder):
    for file in files:
        if file.endswith('.nii.gz') and 'mask' not in file:
            nifti_path = os.path.join(root, file)
            json_path = nifti_path.replace('.nii.gz', '.json')
            
            # Load the NIfTI image
            img = nib.load(nifti_path)
            
            # Flip the axis
            flipped_img = flip_axis(img)
            
            # Create the same folder structure in the output folder
            relative_path = os.path.relpath(root, input_folder)
            subject_folder = os.path.join(output_folder, relative_path)
            os.makedirs(subject_folder, exist_ok=True)
            
            # Save the flipped image
            flipped_nifti_path = os.path.join(subject_folder, os.path.basename(nifti_path).replace('.nii.gz', '_flipped.nii.gz'))
            nib.save(flipped_img, flipped_nifti_path)
            
            # Load and save the JSON metadata
            if os.path.exists(json_path):
                with open(json_path, 'r') as f:
                    metadata = json.load(f)
                flipped_json_path = os.path.join(subject_folder, os.path.basename(json_path).replace('.json', '_flipped.json'))
                with open(flipped_json_path, 'w') as f:
                    json.dump(metadata, f, indent=4)

Prepare naming for nnUNet

In [3]:
import os
import shutil

input_folder = '/home/jaimebarranco/Desktop/tmp/test_inference_flipped_images/flipped_images'
output_folder_nnUNet = '/home/jaimebarranco/Desktop/tmp/test_inference_flipped_images/flipped_images_nnUNet/input'

# Ensure the output folder for nnUNet exists
os.makedirs(output_folder_nnUNet, exist_ok=True)

# Iterate through the input folder to find NIfTI files and rename them
for root, dirs, files in os.walk(input_folder):
    for file in files:
        if file.endswith('.nii.gz'):
            nifti_path = os.path.join(root, file)
            
            # Rename the file with '_0000' before the '.nii.gz'
            new_filename = file.replace('.nii.gz', '_0000.nii.gz')
            new_nifti_path = os.path.join(output_folder_nnUNet, new_filename)
            
            # Copy the file to the new location with the new name
            shutil.copy(nifti_path, new_nifti_path)


Run nnUNet segmentation on the flipped images

In [ ]:
sudo docker run --gpus device=0 --shm-size=10gb --entrypoint=/bin/bash -v /home/jaimebarranco/Desktop/nnUNet:/opt/nnunet_resources -e nnUNet_preprocessed=/opt/nnunet_resources/nnUNet_preprocessed -e nnUNet_raw_data_base=/opt/nnunet_resources/nnUNet_raw_data_base -e RESULTS_FOLDER=/opt/nnunet_resources/nnUNet_trained_models -v /home/jaimebarranco/Desktop/tmp/test_inference_flipped_images/flipped_images_nnUNet:/tmp jaimebarran/fw_gear_aeye:0.0.1 -c "nnUNet_predict -i /tmp/input -o /tmp/output -tr nnUNetTrainerV2 -ctr nnUNetTrainerV2CascadeFullRes -m 3d_fullres -p nnUNetPlansv2.1 -t Task313_Eye"

Flip back the output segmentations

In [ ]:
import os
import nibabel as nib

input_folder = '/home/jaimebarranco/Desktop/tmp/test_inference_flipped_images/flipped_images_nnUNet/output'
output_folder = '/home/jaimebarranco/Desktop/tmp/test_inference_flipped_images/flipped_images_nnUNet/output_flipped_back'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Function to flip the axis of the image back to the original orientation
def flip_axis_back(img):
    data = img.get_fdata()
    flipped_back_data = data[::-1, :, :]
    flipped_back_img = nib.Nifti1Image(flipped_back_data, img.affine, img.header)
    return flipped_back_img

# Iterate through the input folder to find NIfTI files
for root, dirs, files in os.walk(input_folder):
    for file in files:
        if file.endswith('.nii.gz'):
            nifti_path = os.path.join(root, file)
            
            # Load the NIfTI image
            img = nib.load(nifti_path)
            
            # Flip the axis back
            flipped_back_img = flip_axis_back(img)
            
            # Create the same folder structure in the output folder
            relative_path = os.path.relpath(root, input_folder)
            subject_folder = os.path.join(output_folder, relative_path)
            os.makedirs(subject_folder, exist_ok=True)
            
            # Save the flipped back image
            flipped_back_nifti_path = os.path.join(subject_folder, os.path.basename(nifti_path).replace('.nii.gz', '_flipped_back.nii.gz'))
            nib.save(flipped_back_img, flipped_back_nifti_path)